In [1]:
import os
import sys
import json
import numpy as np
import traceback

import pandas as pd

project_dir = '/mnt/d/projects/bassline_transcription'
sys.path.insert(0, project_dir)

from utilities import get_directories, read_metadata, print_midi_array, print_symbolic_representation, load_numpy_midi

from transcription import decode_NN_output
from MIDI_output import create_MIDI_file

## Read Track Metadata

In [2]:
directories = get_directories(project_dir+'/directories.json')

_, track_dicts, track_titles = read_metadata(directories)

midi_dir = directories['midi']['generated']

## NN Output 2 MIDI File

In [3]:
# read NN outputs 
NN_outputs = pd.read_csv('../data/FirstOut.csv').to_numpy()

In [4]:
for idx, NN_output in enumerate(NN_outputs): 
    
    midi_array = decode_NN_output(NN_output, 8, 120)
    
    # Choose The Note to Transpose
    # midi_array = transpose_to_note(midi_array, Note)
    
    # Choose Bassline BPM
    BPM = 125 
    
    #create_MIDI_file(midi_array, BPM, str(idx), midi_dir)

## Numpy Midi Array to MIDI File

In [ ]:
M = 8
midi_dir = '../data/bassline_transcription/midi_notes/M={}'.format(M)

file_names = next(os.walk(midi_dir))[-1]

In [ ]:
for file_name in file_names:
    
    try:
        midi_array = load_numpy_midi(midi_dir, file_name)
        title = file_name.split('.npy')[0]
        BPM = track_dicts[title]['BPM']
        create_MIDI_file(midi_array, BPM, title, midi_dir)
    except KeyboardInterrupt:
        sys.exit()
        pass
    except IndexError as ie:
        print('\n'+file_name+'\nNo Midi Notes!')
    except UnboundLocalError as ule:
        print('\n{}\nMidi array shape:{}'.format(file_name, load_numpy_midi(file_name).shape))  
    except Exception as ex:
        print('\n'+file_name)
        exception_str = ''.join(traceback.format_exception(etype=type(ex), value=ex, tb=ex.__traceback__))
        print(exception_str)

## Old

def find_repetitions(midi_array):
    
    # find where the repetitions occur (from 100)
    lst = []
    first, prev = None, None
    for idx, val in enumerate(midi_array):

        if val == 100 and idx > 0 and prev == None:
            prev = midi_array[idx-1]
            first = idx
        elif first != None and prev != None and val != 100:
            second = idx-1
            lst.append([prev, first, second])
            first = None
            prev = None         
            
    return lst
    
for row in lst:
    
    midi_val = row[0]
    note_len = (row[2] - row[1] +1) + 1
    start_idx = row[1] - 1
       
    print('Note : {} Start idx: {}, length : {}'.format(midi_val, start_idx, note_len))


def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)

def find_repetition_indices(seq):

    repetition_indices = np.where(seq==100)[0]
    x = np.insert(repetition_indices, 0,repetition_indices[0])
    
    return repetition_indices[np.diff(x) == 1]